## Expanding the CIP netCDFfiles and adding data from nav files



In [6]:
import xarray as xr # read netcdf-files
import glob # allows for wildcards in filemanagement
import os #get a list of all directories/files
import re #regex

from scripts.func_nc import nc_save_with_check
from scripts.update_cip_nc import standardize_cip_netcdf

ModuleNotFoundError: No module named 'scripts'

In [108]:
# sample rate to process (current possibilities: 1,5,12 sek)
sample_rate = 1

In [109]:
# -- Paths to datafiles
# Local disk path to nav data:
nav_main_path = '../2022-islas/' # directory with flight data
nav_file_struct_tdyn = '/*_TDYN_*.nc' # structure of nav TDYN file names
nav_file_struct_nav = '/*_NAV_*.nc' # structure of nav NAV file names
drop_flights = ['as220005','as220006'] # flights to drop, (if not all are to be analysed 5 and 6 is in france)
    
# Local disk path to SODA processed CIP files
cip_main_path = f'../Results_2022-islas/{sample_rate}sAveraging/'
cip_file_struct = '/*CIP.nc'

# Save file path
save_path = '../Results_2022-islas/Processed/CIP_processed/'

In [110]:
# -- Get foldernames that are flights (valid in both nav_main_path and cip_main_part)    
# regex for only using folders that are flights
patt = re.compile(r"as2200\d{2}") # flights have the pattern as2200 + 2 digits
     
flights = [
    f for f in os.listdir(nav_main_path) 
    if os.path.isdir(os.path.join(nav_main_path, f)) and patt.fullmatch(f)
    ]
     
# remove flights to drop using listcomprehension
flights = [i for i in flights if i not in drop_flights] # list of folders/flights

# safireid-islasid dictionary
safire_to_islas = {
    'as220007':'IS22-02',
    'as220008':['IS22-03','IS22-04'],
    'as220009':'IS22-05',
    'as220010':'IS22-06',
    'as220011':'IS22-07',
    'as220012':'IS22-08',
    'as220013':'IS22-09',
    'as220014':'IS22-10',
    'as220015':'IS22-11'
    }

In [111]:
# Process all flights in the folders
for flight in flights:
    # ---- Get CDP and NAV data from flight
    if flight == 'as220008':
        nav_file = glob.glob(nav_main_path + flight + nav_file_struct_tdyn) 
        # flight with safireid as 220008 has two islasids and must be addressed separately:
        for islasid in safire_to_islas['as220008']:
            cip_file = glob.glob(cip_main_path + flight + f'/{islasid}' + cip_file_struct) 
            print(f'Reading: {cip_file[0]} and {nav_file[0]}')
            cip_updated_xds = standardize_cip_netcdf(cip_file[0], nav_file[0], islasid)

            # save to new netCDF
            savefile = save_path + f'CIP_update_{sample_rate}s_{islasid}.nc'
            nc_save_with_check(savefile, cip_updated_xds)

            
    else:
        nav_file = glob.glob(nav_main_path + flight + nav_file_struct_tdyn) 
        cip_file = glob.glob(cip_main_path + flight + cip_file_struct) # returns a list, must access with file[0]
        #cip_xds = xr.open_dataset(cip_file) # returns an xarray dataset
        print(f'Reading: {cip_file[0]} and {nav_file[0]}')
        islasid = safire_to_islas[flight]
        cip_updated_xds = standardize_cip_netcdf(cip_file[0], nav_file[0], islasid)

        #Save to new netCDF
        savefile = save_path + f'CIP_update_{sample_rate}s_{islasid}.nc'      
        nc_save_with_check(savefile, cip_updated_xds)

Reading: ../Results_2022-islas/1sAveraging/as220014/04032022_071513_CIP.nc and ../2022-islas/as220014/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220403_as220014_L1_V1.nc
Saving to ../Results_2022-islas/Processed/CIP_processed/CIP_update_1s_IS22-10.nc
Reading: ../Results_2022-islas/1sAveraging/as220009/03262022_080110_CIP.nc and ../2022-islas/as220009/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220326_as220009_L1_V1.nc
Saving to ../Results_2022-islas/Processed/CIP_processed/CIP_update_1s_IS22-05.nc
Reading: ../Results_2022-islas/1sAveraging/as220010/03262022_145034_CIP.nc and ../2022-islas/as220010/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220326_as220010_L1_V1.nc
Saving to ../Results_2022-islas/Processed/CIP_processed/CIP_update_1s_IS22-06.nc
Reading: ../Results_2022-islas/1sAveraging/as220007/03222022_113721_CIP.nc and ../2022-islas/as220007/ISLAS_SAFIRE-ATR42_CORE_TDYN_1HZ_20220322_as220007_L1_V1.nc
Saving to ../Results_2022-islas/Processed/CIP_processed/CIP_update_1s_IS22-02.nc
Reading: ../Results_2022-isl